In [1]:
! pip install -e ..

Obtaining file:///Users/s2341012/Library/CloudStorage/Dropbox/DISMaL_chapter/DISMaL
  Preparing metadata (setup.py) ... done
  Attempting uninstall: dismal
    Found existing installation: dismal 0.1
    Uninstalling dismal-0.1:
      Successfully uninstalled dismal-0.1
  Running setup.py develop for dismal


In [2]:
from dismal.infer import DivergenceModel
from dismal.demography import Epoch
import numpy as np
import matplotlib.pyplot as plt
from dismal.markov_matrices import TransitionRateMatrix, StochasticMatrix

In [3]:
S = [np.ones(10)]*3

In [4]:
mod = DivergenceModel(epochs=3, allow_migration=True)

In [5]:
mod.log_likelihood_3_epoch(params=[3, 2, 4, 3, 6, 8, 4, 0.2/4, 0.4/2, 0.04/6, 0.08/3], S=np.ones(shape=(3,10)))

[[9.99998844e-01 2.42735594e-01 5.89152024e-02 1.42940181e-02
  3.46388455e-03 8.36945671e-04 2.01015934e-04 4.77781815e-05
  1.11760798e-05 2.55743911e-06]
 [9.99986375e-01 2.04145648e-01 4.16467271e-02 8.47622879e-03
  1.71512729e-03 3.43081704e-04 6.73386695e-05 1.28630484e-05
  2.37298821e-06 4.20112816e-07]
 [9.99848164e-01 8.19054553e-02 6.67900133e-03 5.38366829e-04
  4.24488949e-05 3.23502721e-06 2.35676227e-07 1.62733557e-08
  1.05911588e-09 6.47788074e-11]]
nan
[[9.99998844e-01 2.42735594e-01 5.89152024e-02 1.42940181e-02
  3.46388455e-03 8.36945671e-04 2.01015934e-04 4.77781815e-05
  1.11760798e-05 2.55743911e-06]
 [9.99986375e-01 2.04145648e-01 4.16467271e-02 8.47622879e-03
  1.71512729e-03 3.43081704e-04 6.73386695e-05 1.28630484e-05
  2.37298821e-06 4.20112816e-07]
 [9.99848164e-01 8.19054553e-02 6.67900133e-03 5.38366829e-04
  4.24488949e-05 3.23502721e-06 2.35676227e-07 1.62733557e-08
  1.05911588e-09 6.47788074e-11]]
nan
[[9.99998844e-01 2.42735594e-01 5.89152024e-02 1

/Users/s2341012/Library/CloudStorage/Dropbox/DISMaL_chapter/DISMaL/dismal/infer.py:165: RuntimeWarning: invalid value encountered in scalar power
  A[lmbda_idx, sval] = ((lmbda/(lmbda+1) * (1/(lmbda+1)))


nan

In [6]:
params=[3, 2, 4, 3, 6, 8, 4, 0.2/4, 0.4/2, 0.04/6, 0.08/3]
S=np.ones(shape=(3,10))

In [7]:
thetas = params[0:4]
tau1 = params[5]
v = params[6]
tau0 = params[5] + params[6]
Ms_iter = iter(params[7:])

In [8]:
epochs = [Epoch(id=0, allow_migration=False), Epoch(id=1, allow_migration=True), Epoch(id=2, allow_migration=True)]

In [9]:
Q1 = TransitionRateMatrix(thetas=thetas[0:2], ms=[next(Ms_iter) for _ in range(epochs[1].n_m_params)])
Q2 = TransitionRateMatrix(thetas=thetas[2:4], ms=[next(Ms_iter) for _ in range(epochs[2].n_m_params)])

GG = -Q1.eigenvectors_inv @ np.diag(Q1.eigenvectors[:,-1])
CC = -Q2.eigenvectors_inv @ np.diag(Q2.eigenvectors[:,-1])

P1 = StochasticMatrix(Q1.eigenvectors, Q1.eigenvectors_inv, Q1.eigenvalues, t=tau1)
P2 = StochasticMatrix(Q2.eigenvectors, Q2.eigenvectors_inv, Q2.eigenvalues, t=v)


In [44]:
eigvals = [-Q1.eigenvalues[0:3], -Q2.eigenvalues[0:3], np.array([1/params[0]])]
times = [0, tau1, tau0, None]

In [45]:
A, B, C = [DivergenceModel.eigenvalue_transform(lmbdas=eigvals[i], s=S[2], start_time=times[i], end_time=times[i+1])
                                 for i in range(3)]

In [54]:
state_idx = 2
#(np.transpose(GG[state_idx, 0:3]) @ A )
# + P1[state_idx, 0:3] @ np.transpose(CC[0:3, 0:3]) @ B
#+ (1 - ((P1.matrix @ P2.matrix)[state_idx, -1])) * C

array([[5.26990571e-008, 1.81746537e-046, 3.14484759e-085,
        3.64167718e-124, 3.17619375e-163, 2.22666597e-202,
        1.30772556e-241, 6.62225179e-281, 2.95401850e-320,
        0.00000000e+000]])

In [50]:
np.transpose(GG[state_idx, 0:3])

array([-0.0327304 , -0.26258613,  1.29531653])